In [28]:
# Import modules
import ee
import geemap
import geopandas as gpd
import numpy as np

In [29]:
# Initialize Earth Engine
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()

In [ ]:
def get_vis_params(image, aoi_ee):
    # reduceRegion to get the minimum carbon_organic in the AOI
    min_carbon_organic = image.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=aoi_ee,
        scale=3000,
        maxPixels=9e8
    )

    # reduceRegion to get the maximum carbon_organic in the AOI
    max_carbon_organic = image.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=aoi_ee,
    scale=3000,
    maxPixels=9e8
    )

    # Get the result and print the minimum value
    try:
        carbon_organic_min_max = [min_carbon_organic.getInfo()['mean_0_20'], 
                            max_carbon_organic.getInfo()['mean_0_20']]
        vis_params = {'palette': ['darkred', 'red', 
                          'white', 'lightblue', 'blue', 'darkblue'],
               'min': carbon_organic_min_max[0], 'max': carbon_organic_min_max[1]}
        
    except KeyError:
        carbon_organic_min_max = [min_carbon_organic.getInfo()['constant'], 
                            max_carbon_organic.getInfo()['constant']]
        vis_params = {'palette': ['darkred', 'yellow', 'green', 'darkgreen'],
                'min': carbon_organic_min_max[0], 'max': carbon_organic_min_max[1]}
    
    return vis_params

In [ ]:
def get_carbon_organic_image(aoi_ee):
    # Import carbon_organic dataset from GEE
    raw_carbon_organic_dataset = ee.Image("ISDASOIL/Africa/v1/carbon_organic") \
            .select('mean_0_20').clip(aoi_ee)

    carbon_organic_dataset = raw_carbon_organic_dataset.divide(10).exp().subtract(1)

    return carbon_organic_dataset


In [ ]:
# Get ranked carbon_organic suitability
def get_carbon_organic_suitability_image(crop, aoi_ee):
    image_dataset = get_carbon_organic_image(aoi_ee)
    
    suitability_ranges = {
        'Soybean':    [(6, None, 3), (4, 6, 2), (2, 4, 1)],
        'Groundnut':  [(6, None, 3), (4, 6, 2), (2, 4, 1)],
        'Sugarbean':  [(6, None, 3), (4, 6, 2), (2, 4, 1)],
        'Navybean':   [(6, None, 3), (4, 6, 2), (2, 4, 1)],
        'Maize':      [(6, None, 3), (4, 6, 2), (2, 4, 1)],
        'Cowpea':     [(6, None, 3), (4, 6, 2), (2, 4, 1)],
        'Sunflower':  [(6, None, 3), (4, 6, 2), (2, 4, 1)],
    }

    # Standardize crop name input
    crop = crop.strip().replace(" ", "").title()
    ranges = suitability_ranges[crop]

    # Default suitability of 0 (no suitability)
    suitability_image = ee.Image(0).clip(aoi_ee)

    # Apply suitability levels
    for (min_val, max_val, score) in ranges:
        if max_val is None:
            mask = image_dataset.gte(min_val)
        else:
            mask = image_dataset.gte(min_val).And(image_dataset.lt(max_val)) 

        suitability_image = suitability_image.where(mask, score)

    return suitability_image

In [ ]:
# Open source table as a dataframe
# country_gpkg = r"data\zambia_boundary.gpkg"
country_gpkg = r"C:\Users\jedid\Documents\Code\github\crop_soil_mcda\data\zambia_boundary.gpkg"
country_gdf = gpd.read_file(country_gpkg)

# Preview the data
# country_gdf.head()

In [ ]:
# Convert county from gdf to ee
country_ee = geemap.gdf_to_ee(country_gdf)

# Specify crop
crop = "Groundnut"

In [ ]:
# Get carbon_organic image
carbon_organic_dataset = get_carbon_organic_image(country_ee)

# Get ranked crop suitable average carbon_organic
ranked_suitable_carbon_organic = get_carbon_organic_suitability_image(crop, country_ee)

In [34]:
# carbon_organic visualisastion parameters
carbon_organic_vis_params = get_vis_params(carbon_organic_dataset, country_ee)
ranked_carbon_organic_vis_params = get_vis_params(ranked_suitable_carbon_organic, country_ee)

In [35]:
# Visualise layers
m = geemap.Map()
m.zoom_to_gdf(country_gdf)

m.add_layer(country_ee, name= 'Zambia')

m.add_ee_layer(carbon_organic_dataset, vis_params= carbon_organic_vis_params, name= "carbon_organic")
m.add_colorbar(carbon_organic_vis_params, orientation='vertical',  
               label= "carbon_organic", layer_name= f"carbon_organic")

m.add_ee_layer(ranked_suitable_carbon_organic, vis_params= ranked_carbon_organic_vis_params, name= f"{crop} carbon_organic Suitability")
# m.add_colorbar(ranked_carbon_organic_vis_params, orientation='vertical',  
            #    label= "carbon_organic Suitability", layer_name= f"{crop} carbon_organic Suitability")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…